In [1]:
import pandas as pd

In [6]:
file_id = '130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7'
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
df = pd.read_csv(download_url)

In [8]:
print(df.shape)
print(df.dtypes)
print(df.memory_usage(deep=True).sum() / 1024**2, 'MB')

(23368, 25)
Unnamed: 0                    int64
ID  объявления                int64
Количество комнат            object
Тип                          object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Телефоны                     object
Описание                     object
Ремонт                       object
Площадь комнат, м2           object
Балкон                       object
Окна                         object
Санузел                      object
Можно с детьми/животными     object
Дополнительно                object
Название ЖК                  object
Серия дома                   object
Высота потолков, м          float64
Лифт                         object
Мусоропровод                 object
Ссылка на объявление         object
dtype: object
70.47252750396729 MB


In [17]:
missing = df.isna().mean().sort_values(ascending=False)
dups = df.duplicated().sum()
const_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
print('Пропуски, доля:\n', missing.head(10))
print(f'Дубликатов: {dups}.')
print(f'Константные: {const_cols}.')

Пропуски, доля:
 Серия дома                  0.907438
Название ЖК                 0.749743
Парковка                    0.574161
Высота потолков, м          0.520455
Мусоропровод                0.450274
Площадь комнат, м2          0.381291
Балкон                      0.341407
Окна                        0.282994
Можно с детьми/животными    0.260870
Лифт                        0.235365
dtype: float64
Дубликатов: 0.
Константные: ['Тип'].


In [22]:
def find_cat(data):
    for_dummy = []
    not_for_dummy = []

    for name in data.columns:
        col = data[name]
        if col.dtype == 'object':
            if col.nunique() <= 3:
                for_dummy.append(name)
            else:
                not_for_dummy.append(name)

    return for_dummy, not_for_dummy


for_dummy, not_for_dummy = find_cat(df)

print('Можно просто закодировать: ',for_dummy)
print('\nНельзя так просто закодировать: ', not_for_dummy)

Можно просто закодировать:  ['Тип', 'Окна', 'Можно с детьми/животными', 'Мусоропровод']

Нельзя так просто закодировать:  ['Количество комнат', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Санузел', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Лифт', 'Ссылка на объявление']


KeyError: np.int64(0)